In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True) # Round the results to 3 decimal place

In [2]:
def householderHessenberg(matrix):
    A = matrix.astype('float')
    num_rows, num_columns = matrix.shape
    I = np.identity(num_rows)
    A_hut = A.copy() # initial hessenberg matrix
    householder = []
    for column in range(num_columns-2):
        x = np.vstack((np.zeros((column+1,1)), A_hut[column+1:, column].reshape(-1,1))) # append zero above the target row
        # change the operation depending on the sign
        if np.sign(x[column+1]) == 0 or np.sign(x[column+1]) == 1:
            x[column+1] += np.linalg.norm(x)
        else:
            x[column+1] -= np.linalg.norm(x)
        u = x/np.linalg.norm(x) # normalize
        H = I - 2*u@u.T # householder reflection
        A_hut = H @ A_hut @ H
        print(f'H{column+1}')
        print(H)
        householder.append(H)
    return A_hut, householder
    

In [3]:
# matrix = np.array([[1, 0, 2, 3], [-1, 0, 5, 2], [2, -2, 0, 0], [2, -1, 2, 0]]) # sample matrix
matrix = np.random.randint(0, 10, size=(4, 4))
print('Original Matrix\n', matrix)

Original Matrix
 [[0 4 4 7]
 [9 0 4 2]
 [0 4 4 3]
 [2 8 5 0]]


In [4]:
hess, householder = householderHessenberg(matrix)

H1
[[ 1.     0.     0.     0.   ]
 [ 0.    -0.976  0.    -0.217]
 [ 0.     0.     1.     0.   ]
 [ 0.    -0.217  0.     0.976]]
H2
[[ 1.     0.     0.     0.   ]
 [ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.518 -0.856]
 [ 0.     0.    -0.856  0.518]]


In [5]:
print('Hessenberg Matrix\n', hess)

Hessenberg Matrix
 [[ 0.    -5.423 -7.175 -0.334]
 [-9.22   2.118  3.891  3.477]
 [ 0.     8.8    2.212  5.095]
 [ 0.     0.     3.143 -0.33 ]]


In [6]:
def rotation(a, b): # calculate the c and s values in the rotation matrix
    r = np.sqrt(a**2 + b**2)
    c = a/r
    s = -b/r
    return c, s

In [7]:
def givensRotation(hess):
    A = hess.copy()
    shape = A.shape[0]
    givens = []
    Q = np.identity(shape)
    for i in range(shape-1):
        c, s = rotation(A[i, i], A[i+1, i])
        G = np.identity(shape)
        G[i:i+2, i:i+2] = np.array([[c, -s], [s, c]]) # include the rotation in the identity matrix
        A = G @ A # rotate the current matrix to make the subdiagonal zero
        givens.append(G) # rotation matrices
    for rotate in reversed(givens): # calculate the Q matrix by multiplying the rotations in reversed order
        Q = Q @ rotate
    return Q.T, A # transpose Q is Rotation and (Rotation)(Hessian) = R

In [28]:
def qrAlgorithm(hess, tolerance=1e-6, iteration=1000, sigma=0):
    shape = hess.shape[0]
    A_k = hess.copy()
    for k in range(iteration):
        # Computes the QR matrix from Hessenberg using Givens Rotation
        Q, R = givensRotation(A_k - sigma*np.identity(shape)) # sigma is the initial eigenvalue guess
        A_k = R @ Q + sigma*np.identity(shape)
        if np.max(np.abs(np.diag(A_k, k=-1))) < tolerance: # program will terminate when the subdiagonal reached the tolerance
            print('Tolerance reached')
            break
        if k == iteration-1: # program will terminate when max iteration is reached
            print('Max iteration reached')
    return A_k

In [29]:
tolerance = 1e-6
A_k = qrAlgorithm(hess, tolerance)
print('Schur Matrix\n', A_k)

Max iteration reached
Schur Matrix
 [[13.777 -0.331 -0.54  -3.739]
 [ 0.    -4.795 -3.381 -1.062]
 [-0.     5.583 -5.849 -3.183]
 [-0.     0.     0.     0.866]]


To get the eigenvalue of a 2 by 2 matrix:
$$
A_k = 
\begin{bmatrix}
a & b\\
c & d
\end{bmatrix}
$$
$$
A_k - \lambda I = 0
$$
$$
\begin{bmatrix}
a-\lambda & b\\
c & d-\lambda
\end{bmatrix}
= 0
$$
Getting the Determinant:
$$
(a-\lambda)(d-\lambda) - bc = 0
$$
$$
\lambda^2 + (-a-d) \lambda + (ad-bc) = 0
$$
Using the quadratic formual:
$$
\lambda = \frac{-(-a-d) \pm \sqrt{(-a-d)^2+4(ad-bc)}}{2}
$$

In [76]:
def getEigenvalues(A_k, tolerance):
    temp_eig = np.diag(A_k).astype('complex') # diagonal elements
    sub_diag = np.diag(A_k, k=-1) > tolerance # get the subdiagonal with value higher than tolerance
    for i, d in enumerate(sub_diag):
        if d == True:
            a = A_k[i, i]
            b = A_k[i, i+1]
            c = A_k[i+1, i]
            d = A_k[i+1, i+1]
            eig = (-(-a-d) + np.sqrt((-a-d)**2 - 4*(a*d-b*c), dtype='complex')) / 2
            # l = (a-c)/2
            # if l >= 0:
            #     eig = c - b**2 / (l + np.sqrt(l**2 + b**2))
            # else:
            #     eig = c + b**2 / (l + np.sqrt(l**2 + b**2))
            temp_eig[[i, i+1]] = eig, np.conjugate(eig) # replace with the eigenvalue conjugate pair
    return temp_eig
    

In [77]:
print('Calculated Eigenvalues\n', getEigenvalues(A_k, tolerance))
print('Numpy Eigenvalues of the Original Matrix\n', np.linalg.eig(matrix)[0])

Calculated Eigenvalues
 [13.777+0.j    -5.322+4.313j -5.322-4.313j  0.866+0.j   ]
Numpy Eigenvalues of the Original Matrix
 [13.777+0.j    -5.322+4.313j -5.322-4.313j  0.866+0.j   ]
